# Building an Orchid Genus Classifier App Using fastai, Render and Flutter - Part 2

- toc: false
- branch: master
- badges: true
- comments: true
- author: Mike Fuller
- categories: [fastai, jupyter, render, flutter]
- image: images/orchid.jpeg

In this blog series I'll show how to train an orchid genus classifier using [fastai](https://www.fast.ai/), deploy this to [Render](https://render.com/) and create a Flutter app for the front-end. This will be done in two parts:

1. [Dataset Collection and fastai Image Classifier Training](https://mikful.github.io/blog/fastai/jupyter/render/flutter/2020/09/15/orchid-classifier-fastai-render-flutter.html)
2. Render Deployment and Flutter App

This is part 2, please see the associated [github repo for render](https://github.com/mikful/orchid-classifier-render) and [github repo for flutter](https://github.com/mikful/orchid-classifier-flutter) for further implementation details.

## Render Deployment

Now that we've trained our model, we need to deploy our fastai Learner to a dockerized environment, such that we can perform inference on new images. The fastai course v3 had a [starter package](https://github.com/render-examples/fastai-v3) for Render that we'll update to work with the latest fastai library (version 2) and also for fastapi in this case. (Note: the fastbook and fastai website now give different options, such as a binder-hosted jupyter notebook for simple inference, or [seemeai](https://course.fast.ai/deployment_seeme_ai), although I find render and heroku are still very good options and more flexible than binder.)

### requirements.txt

First, let's update our package dependencies (that we listed within our Notebook environment in Part 1 of this blog series). Be sure to use the cpu versions of PyTorch versions within the Notebook, not the larger cuda wheels. You can ensure to set these using `+cpu`.

In [ ]:
torch==1.6.0+cpu 
torchvision==0.7.0+cpu
-f https://download.pytorch.org/whl/torch_stable.html

These will be installed when the Dockerfile builds the docker container image using the following line:

``RUN pip install --upgrade -r requirements.txt``

### server.py

Now we need to update the server.py file that contains the app. Firstly, the original [Starlette](https://www.starlette.io/) asyncio library will be replaced by the [fastapi](https://fastapi.tiangolo.com/) library, which is a little clearer in its syntax and easier to use.

Now we define the location of our exported fastai Learner. In part 1, we saw how to upload this to a Google Cloud Storage bucket - we need to ensure the permissions on the file are set such that we can download this also, which can be done in the file settings in the bucket (for more details [see here](https://cloud.google.com/storage/docs/access-control/making-data-public)).


```python
path = Path(__file__).parent
export_file_url = 'https://storage.googleapis.com/fastai-export-bucket/export.pkl' # google cloud bucket
export_file_name = 'export.pkl'
```

**Inference**

The main functions within the server.py file are fairly self evident, in that it downloads the `export.pkl` file and then loads it into a new fastai `Learner`.

For the inference, the main function is contained within the following code section that takes the bytes from the post request and performs the inference on it. The prediction is then returned in the JSON response.

```python
@app.post("/analyze")
async def analyze(file: bytes = File(...)):
    img_bytes = BytesIO(file)
    prediction, idx, preds = learn.predict(img_bytes.getvalue())
    return JSONResponse({'result': str(prediction)})
```

**HTML**

Note that within the repo is also the css and [html file](https://github.com/mikful/orchid-classifier-render/blob/master/app/view/index.html) for a webapp, that can be used as the interface to perform the inference if desired. However, in this case, I wanted to make a full Android app, not a webapp so setup but didn't use the webpage.

## Flutter App

Flutter is a cross-platform app development solution, with near native application speeds. 

Thanks to the code given at: https://github.com/dnmanveet/Fruit_classifier_app I could successfully connect my render backend to a working Android Flutter App.

It was really as simple as changing the base render deployment location within [main.dart](https://github.com/mikful/orchid-classifier-flutter/blob/6fa185dd2a61e5911d67be69ee6c38c8d46c8ba7/orchid-classifier-flutter/lib/main.dart) to the one previously deployed.

```python
    String base =
        "https://orchid-classifier.onrender.com";
```

Then, all that remained were some interface and styling tweaks and I had a pretty decent looking basic app for trying out on my own images!

## Testing

Below is a gif of some testing I did with some photos I had previously taken. In reality the app worked impressively well, correctly predicting all

























In [ ]:
Note, tha